In [1]:
import math
import re
import pandas as pd
import numpy as np
from collections import Counter
from copy import deepcopy
from pprint import pprint

In [26]:
class Sensor:
    def __init__(self, string):
        self.x, self.y, self.beacon_x, self.beacon_y = [int(coord) for coord in re.findall(r'[xy]=(-?\d+)', string)]
        self.manhattan = abs(self.x - self.beacon_x) + abs(self.y - self.beacon_y)

    def __str__(self):
        return f"({self.x}, {self.y}): r={self.manhattan}"

In [27]:
def naive_merge_ranges(ranges, sensors):
    # Create sets from the given ranges and merge them
    numbers = set()
    for r in ranges:
        numbers |= set(range(r[0], r[1] + 1))

    # Remove beacons
    for sensor in sensors:
        if sensor.beacon_y == 2_000_000 and sensor.beacon_x in numbers:
            numbers.remove(sensor.beacon_x)
    # Return the number of covered x coordinates on this line
    return len(numbers)

In [29]:
# Part 1

sensors = []

with open('input.txt') as f:
    for line in f.read().strip().split('\n'):
        sensors.append(Sensor(line))

line_ranges = []
for sensor in sensors:
    # Sensor does not reach the line of interest
    if abs(sensor.y - 2_000_000) > sensor.manhattan:
        continue
    # Find the range of x coords on the line of interest
    traverse_x = sensor.manhattan - abs(sensor.y - 2_000_000)
    line_ranges.append((sensor.x - traverse_x, sensor.x + traverse_x))

print(naive_merge_ranges(line_ranges, sensors))

5511201


In [53]:
def has_distress_beacon(ranges):
    """Try to find a gap in the given ranges between 0 and 4 million and return it."""
    end = 0
    for rng in ranges:
        # Return the gap if it's found
        if rng[0] > end:
            return end + 1
        if rng[1] > end:
            end = rng[1]
        # Quit past 4 million
        if end > 4_000_000:
            return False
    # Check if the gap is past the last given range
    if end < 4_000_000:
        return end + 1
    return False

In [54]:
# Part 2

sensors = []

with open('input.txt') as f:
    for line in f.read().strip().split('\n'):
        sensors.append(Sensor(line))

for line in range(4_000_001):
    line_ranges = []
    for sensor in sensors:
        # Sensor does not reach the line of interest
        if abs(sensor.y - line) > sensor.manhattan:
            continue
        # Find the range of x coords on the line of interest
        traverse_x = sensor.manhattan - abs(sensor.y - line)
        line_ranges.append((sensor.x - traverse_x, sensor.x + traverse_x))
    # Find the distress beacon and print its tuning frequency
    if beacon := has_distress_beacon(sorted(line_ranges)):
        print(beacon * 4_000_000 + line)
        break

11318723411840
